# **Import Library dan Data** 

In [145]:
import numpy as np
import pandas as pd

In [146]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
import pandas as pd

lokasi = "/content/drive/MyDrive/Colab Notebooks/Project_Credit Risk Model/lc_2016_2017.csv"
loan_data = pd.read_csv(lokasi)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (18,54) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# **Target Variable dan Missing Value**

In [148]:
# Membaca dan melihat 5 baris pertama
loan_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,112435993,NaN,2300,2300,2300.0,36 months,12.62,77.08,C,C1,...,0.0,NaN,1.0,2.0,2315.0,55.0,7100.0,1.0,0.0,2.0
1,112290210,NaN,16000,16000,16000.0,60 months,12.62,360.95,C,C1,...,11078.0,69.0,3.0,5.0,1962.0,94.0,31900.0,0.0,6.0,1.0
2,112436985,NaN,6025,6025,6025.0,36 months,15.05,209.01,C,C4,...,0.0,NaN,1.0,2.0,1950.0,45.0,27700.0,1.0,5.0,3.0
3,112439006,NaN,20400,20400,20400.0,36 months,9.44,652.91,B,B1,...,53566.0,95.0,1.0,2.0,4240.0,60.0,46900.0,1.0,1.0,1.0
4,112438929,NaN,13000,13000,13000.0,36 months,11.99,431.73,B,B5,...,8466.0,72.0,0.0,1.0,2996.0,78.0,7800.0,0.0,0.0,0.0


In [149]:
# Melihat total baris dan kolom
loan_data.shape

## Total jumlah baris 759,338 dan 72 kolom

(759338, 72)

In [150]:
# Melihat info seluruh kolom
loan_data.info()

## Terdapat beberapa kolom/feature yang memiliki null values atau missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759338 entries, 0 to 759337
Data columns (total 72 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           759338 non-null  int64  
 1   member_id                    0 non-null       float64
 2   loan_amnt                    759338 non-null  int64  
 3   funded_amnt                  759338 non-null  int64  
 4   funded_amnt_inv              759338 non-null  float64
 5   term                         759338 non-null  object 
 6   int_rate                     759338 non-null  float64
 7   installment                  759338 non-null  float64
 8   grade                        759338 non-null  object 
 9   sub_grade                    759338 non-null  object 
 10  emp_title                    708368 non-null  object 
 11  emp_length                   708975 non-null  object 
 12  home_ownership               759338 non-null  object 
 13 

## **Target Varaible**

**Dikarenakan project ini untuk mengetahui bad loan & good loan, maka perlu dibuat feature baru, yaitu target variable yang merepresentasikan bad loan (sebagai 1) dan good loan (sebagai 0)**

In [151]:
# Melihat unique values pada feature loan_status
loan_data.loan_status.unique()

array(['Current', 'Fully Paid', 'In Grace Period', 'Late (31-120 days)',
       'Late (16-30 days)', 'Charged Off', 'Default'], dtype=object)

In [152]:
# Membuat feature baru yaitu good_bad sebagai target variable,
# Jika loan_statusnya 'Charged Off', 'Default', 'Late (31-120 days)', 'Late (16-30 days)' 
# akan dianggap sebagai bad_loan atau 1 dan nilai selain itu akan dianggap good loan atau 0
loan_data['good_bad']=np.where(loan_data.loc[:,'loan_status'].isin(['Late (31-120 days)', 'Late (16-30 days)', 
                                                                'Charged Off', 'Default']),1,0)

In [153]:
# Melihat distribusi 0 dan 1
loan_data.good_bad.value_counts()

0    702875
1     56463
Name: good_bad, dtype: int64

In [154]:
loan_data[['loan_status','good_bad']]

,loan_status,good_bad
0,Current,0
1,Current,0
2,Current,0
3,Current,0
4,Current,0
...,...,...
759333,Current,0
759334,Current,0
759335,Late (16-30 days),1
759336,Current,0


## **Missing Value**

**Feature yang memiliki missing values lebih dari 50% akan di drop, karena jika ingin diisi dengan nilai lain seperti median atau mean, maka errornya akan sangat tinggi. Lebih baik di drop agar tidak membuat model semakin tidak akurat.**

In [155]:
# Menghitung Jumlah Missing Values
missing_values=pd.DataFrame(loan_data.isnull().sum()/loan_data.shape[0]) 

In [156]:
# Memfilter data kosong yang lebih dari 50%
missing_values=missing_values[missing_values.iloc[:,0] > 0.5] 

In [157]:
# Mengurutkan data yang yang kosong dari yang paling tinggi
missing_values.sort_values([0],ascending=False) 

,0
member_id,1.000000
desc,0.999978
dti_joint,0.955223
annual_inc_joint,0.955220
verification_status_joint,0.955220
mths_since_last_record,0.814076
mths_since_last_major_derog,0.716720


In [158]:
#Mengahpus kolom yang berisi kosong lebih dari 50%
loan_data.dropna(thresh=loan_data.shape[0]*0.5, how='all', axis=1, inplace=True) 

In [159]:
missing_values=pd.DataFrame(loan_data.isnull().sum()/loan_data.shape[0])
missing_values=missing_values[missing_values.iloc[:,0] > 0.5]
missing_values.sort_values([0],ascending=False)

,0


# **Data Splitting**

In [160]:
loan_data.shape

(759338, 66)

In [161]:
from sklearn.model_selection import train_test_split

In [162]:
# Membagi data menjadi 80% data latih dan 20% data uji

X=loan_data.drop('good_bad',axis=1)
y=loan_data['good_bad']

X_train, X_test, y_train, y_test=train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

In [163]:
X_train.shape, X_test.shape

((607470, 65), (151868, 65))

In [164]:
y_train.value_counts(normalize=True)

0    0.925642
1    0.074358
Name: good_bad, dtype: float64

In [165]:
# Distribusi y_test sudah sama persis dengan y_train
y_test.value_counts(normalize=True)

0    0.925639
1    0.074361
Name: good_bad, dtype: float64

# **Data Cleaning**

In [166]:
X_train.select_dtypes(include=['object','bool'])

,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,earliest_cr_line,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type
147577,36 months,C,C1,Administrative assistant,4 years,MORTGAGE,Source Verified,Feb-2017,Current,n,debt_consolidation,Debt consolidation,582xx,ND,Aug-1994,w,Dec-2017,Jan-2018,Dec-2017,Individual
505776,36 months,A,A1,director of workforce development,10+ years,RENT,Source Verified,Apr-2016,Fully Paid,n,debt_consolidation,Debt consolidation,112xx,NY,Sep-2001,w,Jul-2017,NaN,Nov-2017,Individual
412425,36 months,B,B1,Nanny,< 1 year,RENT,Not Verified,Jul-2017,Current,n,credit_card,Credit card refinancing,112xx,NY,May-1999,w,Dec-2017,Jan-2018,Dec-2017,Individual
618730,36 months,B,B5,QA Software Analyst,5 years,RENT,Source Verified,Oct-2016,Current,n,credit_card,Credit card refinancing,840xx,UT,Jun-2005,w,Dec-2017,Jan-2018,Dec-2017,Individual
111143,36 months,C,C1,Supervisor,6 years,RENT,Source Verified,Mar-2017,Current,n,debt_consolidation,Debt consolidation,334xx,FL,Jan-2012,w,Dec-2017,Jan-2018,Dec-2017,Individual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150426,36 months,A,A4,Construction Superintendent,8 years,MORTGAGE,Source Verified,Feb-2017,Fully Paid,n,other,Other,913xx,CA,Nov-1998,w,Nov-2017,NaN,Dec-2017,Individual
195135,36 months,B,B2,NaN,NaN,RENT,Verified,Jan-2017,Current,n,home_improvement,Home improvement,320xx,FL,Sep-1993,f,Dec-2017,Jan-2018,Dec-2017,Individual
182511,36 months,A,A3,Aquatics Manager,10+ years,MORTGAGE,Not Verified,Jan-2017,Current,n,home_improvement,Home improvement,988xx,WA,Mar-1988,w,Dec-2017,Jan-2018,Dec-2017,Joint App
82596,36 months,C,C3,making plant planner,3 years,MORTGAGE,Verified,Apr-2017,Current,n,debt_consolidation,Debt consolidation,121xx,NY,Jan-2003,w,Dec-2017,Jan-2018,Dec-2017,Individual


In [167]:
for col in X_train.select_dtypes(include=['object','bool']).columns:
  print(col)
  print(X_train[col].unique())
  print()

term
['36 months' '60 months']

grade
['C' 'A' 'B' 'E' 'D' 'G' 'F']

sub_grade
['C1' 'A1' 'B1' 'B5' 'A4' 'E2' 'A5' 'D1' 'C4' 'C3' 'G3' 'D3' 'E1' 'C5'
 'D2' 'C2' 'A2' 'E3' 'D5' 'E4' 'B2' 'F2' 'B4' 'A3' 'D4' 'B3' 'F1' 'F5'
 'G2' 'E5' 'F3' 'G1' 'G5' 'F4' 'G4']

emp_title
['Administrative assistant' 'director of workforce development' 'Nanny'
 ... 'Operations Planning Manager' 'Sr. xPression Developer'
 'making plant planner']

emp_length
['4 years' '10+ years' '< 1 year' '5 years' '6 years' '7 years' '2 years'
 '3 years' '8 years' '1 year' nan '9 years']

home_ownership
['MORTGAGE' 'RENT' 'OWN' 'ANY' 'NONE']

verification_status
['Source Verified' 'Not Verified' 'Verified']

issue_d
['Feb-2017' 'Apr-2016' 'Jul-2017' 'Oct-2016' 'Mar-2017' 'May-2016'
 'Mar-2016' 'Aug-2017' 'Jul-2016' 'Jan-2017' 'Sep-2016' 'Aug-2016'
 'Nov-2016' 'Dec-2016' 'Jun-2016' 'Sep-2017' 'Feb-2016' 'May-2017'
 'Jun-2017' 'Apr-2017' 'Jan-2016']

loan_status
['Current' 'Fully Paid' 'Late (16-30 days)' 'Charged Off'
 'La

In [168]:
col_need_to_clean=['term','emp_lenght','issue_d','earlist_cr_line','last_pymnt_d',
                   'next_pymnt_d','last_credit_pull_d']

In [169]:
X_train['term'].unique()

array(['36 months', '60 months'], dtype=object)

In [170]:
# Convert data type menjadi numeric 
pd.to_numeric(x_train['term'].apply(str).str.replace('months',''))

147577    36
505776    36
412425    36
618730    36
111143    36
          ..
150426    36
195135    36
182511    36
82596     36
165904    36
Name: term, Length: 607470, dtype: int64

In [171]:
X_train['term']=pd.to_numeric(X_train['term'].apply(str).str.replace('months',''))

In [172]:
# Cek values apa saja yang harus di cleaning
X_train['emp_length'].unique()

array(['4 years', '10+ years', '< 1 year', '5 years', '6 years',
       '7 years', '2 years', '3 years', '8 years', '1 year', nan,
       '9 years'], dtype=object)

In [174]:
X_train['emp_length'] = X_train['emp_length'].apply(str).str.replace('\+ years', '')
X_train['emp_length'] = X_train['emp_length'].apply(str).str.replace(' years', '')
X_train['emp_length'] = X_train['emp_length'].apply(str).str.replace('< 1 year', str(0))
X_train['emp_length'] = X_train['emp_length'].apply(str).str.replace(' year', '')
X_train['emp_length'] = X_train['emp_length'].apply(str).str.replace('nan', str(0))


X_train['emp_length'].fillna(value =0, inplace=True)
X_train['emp_length'] = pd.to_numeric(X_train['emp_length'])

<ipython-input-174-3f2f1ac64688>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  X_train['emp_length'] = X_train['emp_length'].apply(str).str.replace('\+ years', '')


In [175]:
X_train['emp_length'] 

147577     4
505776    10
412425     0
618730     5
111143     6
          ..
150426     8
195135     0
182511    10
82596      3
165904     1
Name: emp_length, Length: 607470, dtype: int64

In [176]:
# Cek feature date
col_date = ['issue_d', 'earliest_cr_line', 'last_pymnt_d',
                    'next_pymnt_d', 'last_credit_pull_d']

X_train[col_date]

,issue_d,earliest_cr_line,last_pymnt_d,next_pymnt_d,last_credit_pull_d
147577,Feb-2017,Aug-1994,Dec-2017,Jan-2018,Dec-2017
505776,Apr-2016,Sep-2001,Jul-2017,NaN,Nov-2017
412425,Jul-2017,May-1999,Dec-2017,Jan-2018,Dec-2017
618730,Oct-2016,Jun-2005,Dec-2017,Jan-2018,Dec-2017
111143,Mar-2017,Jan-2012,Dec-2017,Jan-2018,Dec-2017
...,...,...,...,...,...
150426,Feb-2017,Nov-1998,Nov-2017,NaN,Dec-2017
195135,Jan-2017,Sep-1993,Dec-2017,Jan-2018,Dec-2017
182511,Jan-2017,Mar-1988,Dec-2017,Jan-2018,Dec-2017
82596,Apr-2017,Jan-2003,Dec-2017,Jan-2018,Dec-2017


In [177]:
X_train['issue_d'].unique()

array(['Feb-2017', 'Apr-2016', 'Jul-2017', 'Oct-2016', 'Mar-2017',
       'May-2016', 'Mar-2016', 'Aug-2017', 'Jul-2016', 'Jan-2017',
       'Sep-2016', 'Aug-2016', 'Nov-2016', 'Dec-2016', 'Jun-2016',
       'Sep-2017', 'Feb-2016', 'May-2017', 'Jun-2017', 'Apr-2017',
       'Jan-2016'], dtype=object)

In [178]:
for col in col_date:
  X_train[col]=pd.to_datetime(X_train[col])

In [179]:
  X_train[col_date]

,issue_d,earliest_cr_line,last_pymnt_d,next_pymnt_d,last_credit_pull_d
147577,2017-02-01,1994-08-01,2017-12-01,2018-01-01,2017-12-01
505776,2016-04-01,2001-09-01,2017-07-01,NaT,2017-11-01
412425,2017-07-01,1999-05-01,2017-12-01,2018-01-01,2017-12-01
618730,2016-10-01,2005-06-01,2017-12-01,2018-01-01,2017-12-01
111143,2017-03-01,2012-01-01,2017-12-01,2018-01-01,2017-12-01
...,...,...,...,...,...
150426,2017-02-01,1998-11-01,2017-11-01,NaT,2017-12-01
195135,2017-01-01,1993-09-01,2017-12-01,2018-01-01,2017-12-01
182511,2017-01-01,1988-03-01,2017-12-01,2018-01-01,2017-12-01
82596,2017-04-01,2003-01-01,2017-12-01,2018-01-01,2017-12-01


In [181]:
# Lakukan hal yang sama untuk X_test
X_test['term'] = pd.to_numeric(X_test['term'].apply(str).str.replace(' months', ''))

X_test['emp_length'] = X_test['emp_length'].apply(str).str.replace('\+ years', '')
X_test['emp_length'] = X_test['emp_length'].apply(str).str.replace(' years', '')
X_test['emp_length'] = X_test['emp_length'].apply(str).str.replace('< 1 year', str(0))
X_test['emp_length'] = X_test['emp_length'].apply(str).str.replace(' year', '')
X_test['emp_length'] = X_test['emp_length'].apply(str).str.replace('nan', str(0))

X_test['emp_length'].fillna(value = 0, inplace=True)
X_test['emp_length'] = pd.to_numeric(X_test['emp_length'])


for col in col_date:
    X_test[col] = pd.to_datetime(X_test[col])

<ipython-input-181-a61123c26b7e>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  X_test['emp_length'] = X_test['emp_length'].apply(str).str.replace('\+ years', '')


In [183]:
# Check apakah berhasil di cleaning
X_test[col_date].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151868 entries, 280647 to 267584
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   issue_d             151868 non-null  datetime64[ns]
 1   earliest_cr_line    151868 non-null  datetime64[ns]
 2   last_pymnt_d        151655 non-null  datetime64[ns]
 3   next_pymnt_d        118276 non-null  datetime64[ns]
 4   last_credit_pull_d  151867 non-null  datetime64[ns]
dtypes: datetime64[ns](5)
memory usage: 7.0 MB
